## House Hunting in SF: Venue-based clustering

In [66]:
import pandas as pd
import requests


import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')




Libraries imported.


### Import Redfin housing data

In [61]:
# The code was removed by Watson Studio for sharing.

,ZIP OR POSTAL CODE,ADDRESS,LATITUDE,LONGITUDE
0,94132,257 N Lake Merced Hls,37.712635,-122.481353
1,94112,1957 San Jose Ave,37.725763,-122.442057
2,94134,1328-1330 Silver Ave,37.731009,-122.409813
3,94123,1501 Greenwich St #202,37.800064,-122.424680
4,94118,69 Palm Ave #4,37.784415,-122.458306


In [67]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of SF are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of SF are 37.7790262, -122.4199061.


In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
# create map of SF using latitude and longitude values
map_sf = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, address in zip(sf_zip['LATITUDE'], sf_zip['LONGITUDE'], sf_zip['ADDRESS']):
    label = '{}'.format(address)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_sf)  
    
map_sf

## Define Foursquare Credentials and Version

In [68]:
CLIENT_ID = '4LAQXXPJTQJB10NOMNB1IMEDN4BNT00OIKZ4IE341HN1QYVY' # your Foursquare ID
CLIENT_SECRET = 'KG22Z5PWCGZUCYI5IQXYYRITJ0ITHQBYKSBRZAWNF1JLCLDS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius in meters


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4LAQXXPJTQJB10NOMNB1IMEDN4BNT00OIKZ4IE341HN1QYVY
CLIENT_SECRET:KG22Z5PWCGZUCYI5IQXYYRITJ0ITHQBYKSBRZAWNF1JLCLDS


### Explore Neighbourhoods in SF

In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ADDRESS', 
                  'ADDRESS Latitude', 
                  'ADDRESS Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [70]:
sf_venues = getNearbyVenues(names=sf_zip['ADDRESS'],
                                   latitudes=sf_zip['LATITUDE'],
                                   longitudes=sf_zip['LONGITUDE']
                                  )

257 N Lake Merced Hls
1957 San Jose Ave
1328-1330 Silver Ave
1501 Greenwich St #202
69 Palm Ave #4
878 47th Ave
8 Octavia St #301
400 Beale St #1012
222 Parnassus Ave Unit B
1133 Russia Ave
663 Lakeview Ave
2002 3rd #117
99 Rausch St #113
2166 47th Ave
25 Joice St
1326 Palou Ave
1139 Judah St
1635 29th Ave
520 6th St #15
5090 Diamond Heights Blvd Unit A
2078 45th Ave
1879 19th Ave
3 Mabrey Ct
2954 23rd St
56 Moss St #2
199 Tiffany Ave #403
42 Littlefield Ter
142 Elmira St
1819 30th Ave
1388 Gough St #1101
193 Caine Ave
400 Beale St #1007
53 Vesta St
3301 Cesar Chavez #3305
3301 Cesar Chavez Unit 3301A
1250 Ellis St Unit CALL
3535 Wawona St #318
99 Rausch St #214
72 Lawrence Ave
555 Innes Ave #412
1515 15th St #307
1488 Harrison St #201
1160 Mission St #1907
1029 Natoma St #2
1001 17th St #403
499 33rd Ave #308
1667 Silver Ave
594-598 Moscow St
1731 Oakdale Ave
343-345 Russia Ave
1182 Palou Ave
52 Innes Ct #302
10 Innes Ct #306
10 Innes Ct #203
10 Innes Ct #303
51 Innes Ct #404
10 Innes

In [71]:
print(sf_venues.shape)
sf_venues.head()

(2247, 7)


,ADDRESS,ADDRESS Latitude,ADDRESS Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,257 N Lake Merced Hls,37.712635,-122.481353,Holy Trinity Center,37.714081,-122.483060,Church
1,257 N Lake Merced Hls,37.712635,-122.481353,Lake Merced Run Loop,37.713463,-122.485606,Trail
2,257 N Lake Merced Hls,37.712635,-122.481353,San Francisco Golf Club,37.712459,-122.476884,Golf Course
3,257 N Lake Merced Hls,37.712635,-122.481353,Parkmerced Dog Run,37.715189,-122.479566,Dog Run
4,257 N Lake Merced Hls,37.712635,-122.481353,Lake Merced Fishing Pier And Nature Trail,37.713396,-122.486065,Trail


In [72]:
sf_venues.groupby('ADDRESS').count()

,ADDRESS Latitude,ADDRESS Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
ADDRESS,,,,,,
10 Innes Ct #203,8,8,8,8,8,8
10 Innes Ct #303,15,15,15,15,15,15
10 Innes Ct #306,8,8,8,8,8,8
1001 17th St #403,35,35,35,35,35,35
1029 Natoma St #2,100,100,100,100,100,100
1133 Russia Ave,7,7,7,7,7,7
1139 Judah St,37,37,37,37,37,37
1160 Mission St #1907,100,100,100,100,100,100
1182 Palou Ave,3,3,3,3,3,3


In [73]:
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 293 uniques categories.


### Analyze each Neighborhood

In [74]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
sf_onehot['ADDRESS'] = sf_venues['ADDRESS'] 
# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]
sf_onehot.head()

,ADDRESS,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater

In [75]:
## group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

sf_grouped = sf_onehot.groupby('ADDRESS').mean().reset_index()
sf_grouped

,ADDRESS,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater

In [76]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5
for hood in sf_grouped['ADDRESS']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['ADDRESS'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----10 Innes Ct #203----
               venue  freq
0        Art Gallery  0.25
1                Spa  0.12
2      Grocery Store  0.12
3  Outdoor Sculpture  0.12
4       Home Service  0.12


----10 Innes Ct #303----
               venue  freq
0        Art Gallery  0.27
1    Harbor / Marina  0.13
2      Grocery Store  0.13
3  Outdoor Sculpture  0.13
4       Home Service  0.13


----10 Innes Ct #306----
               venue  freq
0        Art Gallery  0.25
1                Spa  0.12
2      Grocery Store  0.12
3  Outdoor Sculpture  0.12
4       Home Service  0.12


----1001 17th St #403----
            venue  freq
0            Café  0.11
1            Park  0.09
2     Coffee Shop  0.06
3  Breakfast Spot  0.06
4   Deli / Bodega  0.06


----1029 Natoma St #2----
              venue  freq
0      Cocktail Bar  0.06
1         Nightclub  0.05
2  Sushi Restaurant  0.04
3           Gay Bar  0.04
4       Coffee Shop  0.04


----1133 Russia Ave----
            venue  freq
0      Playground  0.14
1    

In [77]:
# put into dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['ADDRESS']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['ADDRESS'] = sf_grouped['ADDRESS']
for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.shape

(56, 6)

## Cluster!

In [79]:
# set number of clusters
kclusters = 6
sf_grouped_clustering = sf_grouped.drop('ADDRESS', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 3, 5, 3, 3, 3, 3], dtype=int32)

In [80]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = sf_zip
sf_merged = sf_merged.merge(neighborhoods_venues_sorted, on = 'ADDRESS')

sf_merged.head()

,ZIP OR POSTAL CODE,ADDRESS,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,94132,257 N Lake Merced Hls,37.712635,-122.481353,3,Trail,Golf Course,Dog Run,Church,Zoo Exhibit
1,94112,1957 San Jose Ave,37.725763,-122.442057,3,Tennis Court,Dessert Shop,Sandwich Place,Light Rail Station,Pool
2,94134,1328-1330 Silver Ave,37.731009,-122.409813,0,Chinese Restaurant,Grocery Store,Vietnamese Restaurant,Donut Shop,Tea Room
3,94123,1501 Greenwich St #202,37.800064,-122.424680,3,Coffee Shop,Spa,Italian Restaurant,Hotel,Cosmetics Shop
4,94118,69 Palm Ave #4,37.784415,-122.458306,3,Coffee Shop,Chinese Restaurant,Thai Restaurant,Bakery,Bank


In [81]:
sf_merged

,ZIP OR POSTAL CODE,ADDRESS,LATITUDE,LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,94132,257 N Lake Merced Hls,37.712635,-122.481353,3,Trail,Golf Course,Dog Run,Church,Zoo Exhibit
1,94112,1957 San Jose Ave,37.725763,-122.442057,3,Tennis Court,Dessert Shop,Sandwich Place,Light Rail Station,Pool
2,94134,1328-1330 Silver Ave,37.731009,-122.409813,0,Chinese Restaurant,Grocery Store,Vietnamese Restaurant,Donut Shop,Tea Room
3,94123,1501 Greenwich St #202,37.800064,-122.424680,3,Coffee Shop,Spa,Italian Restaurant,Hotel,Cosmetics Shop
4,94118,69 Palm Ave #4,37.784415,-122.458306,3,Coffee Shop,Chinese Restaurant,Thai Restaurant,Bakery,Bank
5,94121,878 47th Ave,37.771922,-122.507591,3,Bus Stop,Chinese Restaurant,Grocery Store,Deli / Bodega,Lake
6,94102,8 Octavia St #301,37.772549,-122.423361,3,Wine Bar,Cocktail Bar,Sushi Restaurant,New American Restaurant,Optical Shop
7,94105,400 Beale St #1012,37.786905,-122.390920,3,Coffee Shop,Gym,Food Truck,Gym / Fitness Center,Scenic Lookout
8,94117,222 Parnassus Ave Unit B,37.764716,-122.453196,3,Park,Mexican Restaurant,Organic Grocery,Pharmacy,Breakfast Spot
9,94112,1133 Russia Ave,37.717312,-122.427777,5,Playground,Scenic Lookout,Baseball Field,Soccer Field,Skate Park


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['LATITUDE'], sf_merged['LONGITUDE'], sf_merged['ADDRESS'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(map_clusters)
       
map_clusters

## Cluster 0

In [82]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,1328-1330 Silver Ave,Chinese Restaurant,Grocery Store,Vietnamese Restaurant,Donut Shop,Tea Room
17,1635 29th Ave,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Dim Sum Restaurant,Bubble Tea Shop
21,1879 19th Ave,Chinese Restaurant,Pharmacy,Vietnamese Restaurant,Dessert Shop,Coffee Shop
28,1819 30th Ave,Chinese Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Playground,Coffee Shop
32,53 Vesta St,Chinese Restaurant,Vietnamese Restaurant,Grocery Store,Sandwich Place,Dim Sum Restaurant
46,1667 Silver Ave,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Sandwich Place,Vietnamese Restaurant


## Cluster 1

In [83]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
39,555 Innes Ave #412,Art Gallery,Grocery Store,Restaurant,Spa,Harbor / Marina
51,52 Innes Ct #302,Grocery Store,Restaurant,Harbor / Marina,Outdoor Sculpture,Home Service
52,10 Innes Ct #306,Art Gallery,Spa,Restaurant,Grocery Store,Harbor / Marina
53,10 Innes Ct #203,Art Gallery,Spa,Restaurant,Grocery Store,Harbor / Marina
54,10 Innes Ct #303,Art Gallery,Restaurant,Grocery Store,Harbor / Marina,Outdoor Sculpture
55,10 Innes Ct #303,Art Gallery,Restaurant,Grocery Store,Harbor / Marina,Outdoor Sculpture
56,51 Innes Ct #404,Art Gallery,Grocery Store,Restaurant,Harbor / Marina,Outdoor Sculpture


## Cluster 2

In [84]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,1326 Palou Ave,Gym,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space


## Custer 3

In [85]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,257 N Lake Merced Hls,Trail,Golf Course,Dog Run,Church,Zoo Exhibit
1,1957 San Jose Ave,Tennis Court,Dessert Shop,Sandwich Place,Light Rail Station,Pool
3,1501 Greenwich St #202,Coffee Shop,Spa,Italian Restaurant,Hotel,Cosmetics Shop
4,69 Palm Ave #4,Coffee Shop,Chinese Restaurant,Thai Restaurant,Bakery,Bank
5,878 47th Ave,Bus Stop,Chinese Restaurant,Grocery Store,Deli / Bodega,Lake
6,8 Octavia St #301,Wine Bar,Cocktail Bar,Sushi Restaurant,New American Restaurant,Optical Shop
7,400 Beale St #1012,Coffee Shop,Gym,Food Truck,Gym / Fitness Center,Scenic Lookout
8,222 Parnassus Ave Unit B,Park,Mexican Restaurant,Organic Grocery,Pharmacy,Breakfast Spot
10,663 Lakeview Ave,Playground,Park,Cupcake Shop,Deli / Bodega,Mountain
11,2002 3rd #117,Café,Park,Taco Place,Sandwich Place,Bakery


## Cluster 4

In [86]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,2078 45th Ave,Baseball Field,Bus Station,Trail,Food Truck,Fondue Restaurant


## Cluster 5

In [87]:
sf_merged.loc[sf_merged['Cluster Labels'] == 5, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,ADDRESS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,1133 Russia Ave,Playground,Scenic Lookout,Baseball Field,Soccer Field,Skate Park
47,594-598 Moscow St,Playground,Baseball Field,Basketball Court,Scenic Lookout,Zoo Exhibit
